# Condition Variables

In multithreaded programs, we often want to have one thread wait for some condition to be true before it can continue execution.

We might need one thread wait for some other thread to complete some action before it can continue execution.

We accomplish this using **condition variables**.

A **condition variable** is a queue of waiting threads.

